In [340]:
import csv 
import pandas as pd
# 按照每门课程的考试班级数目进行排序
def SamplingCourse(file):
    with open(file,"r",encoding="UTF-8") as f:
        reader = csv.reader(f)
        column = [row[0] for row in reader]
    hashmapClass = {}
    for i in range(len(column)):
        hashmapClass[column[i]] = hashmapClass.get(column[i],0) + 1
    sortedCourse = sorted(hashmapClass.items(), key=lambda d:d[1], reverse = True)
    return sortedCourse[30:40]

In [341]:
# 生成课程和班级的表
data = pd.read_csv("class.csv", usecols=["ci_course_no","ci_class_name","ci_student_number","ci_teacher_name","ci_class_dep"])

In [342]:
# 找出共同能考试的时间
def findCommonTime(temp, listClass):
    commonTime = []
    for i in range(len(temp)):
        commonTime.append(temp[i] & listClass[i])
    return commonTime

In [343]:
# 将结果写入表中
def writeToCsv(temp, course, classes, number, teacher):
    course = [course] * len(classes)
    time = [temp.index(1)] * len(classes)
    test = pd.DataFrame({'课程': course, "考试时间": time, "班级": classes,"班级人数": number, "监考老师": teacher})
    test.to_csv("temp.csv",encoding = "utf-8",mode="a",index = None, header = False)

In [344]:
# 讲已经排考成功的1修改成0, 如[1,1,1,1]->[0,1,1,1] 维护表
def updateClassTime(dict, listclass, index):
    for classes in listclass:
        dict[classes][index] = 0
    return dict

In [345]:
# 函数的效果是对班级进行统, 目的是每个班级必须有一个考试时间表 
# 比如 "行政管理[193-4]班,人工智能191班" -> ['行政管理193班', '行政管理194班', '人工智能191班']
def unitedClass(strClass):
    listClasses = strClass.split(",")
    splitClassList = []
    for classes in listClasses:
        if "-" in classes:
            middleIndex = classes.index("-")
            firstIndex = classes.index("[")
            lastIndex = classes.index("]")
            lastClassNum = int(classes[middleIndex + 1 :lastIndex])
            firstClassNum = int(classes[middleIndex - 1])
            for i in range( firstClassNum, lastClassNum + 1):
                splitClassList.append(classes[0:firstIndex] + classes[firstIndex + 1:middleIndex-1] + str(i) + "班")
        else:
            splitClassList.append(classes)
    return splitClassList

In [346]:
# 这里是在定义拆分规则
# 可以人数超出阈值, 则拆分班级, 如果人数没有超过, 就不拆
# 可以避免合班的问题
def splitClass(classWhoTakeCourse,classCourseNumber):
    newClassWhoTakeCourse = []
    newClassCourseNumber = []
    for i in range(len(classCourseNumber)):
        # 这个数字怎么订 还是要看教室的具体规模
        if classCourseNumber[i] > 80:
            classes = unitedClass(classWhoTakeCourse[i])
            # 如果拆分出来的班级小于30人, 就没有必要拆
            if classCourseNumber[i] // len(classes) <= 30:
                newClassCourseNumber.append(classCourseNumber[i])
                newClassWhoTakeCourse.append(classWhoTakeCourse[i])
            else:
                for unit in classes:
                    newClassWhoTakeCourse.append(unit)
                    # 这里需要插一段每个班级真正人数的查询
                    # 这里是偷懒取了个平均 
                    newClassCourseNumber.append(classCourseNumber[i]//len(classes))
        else:
            newClassCourseNumber.append(classCourseNumber[i])
            newClassWhoTakeCourse.append(classWhoTakeCourse[i])
    return newClassWhoTakeCourse, newClassCourseNumber

In [347]:
# 建立每个老师和时间的联系
dictTeacherTime = {}
for i in range(len(data)):
    teacher = data.loc[i]["ci_teacher_name"]
    if teacher is np.nan:
        continue
    elif ":" in teacher:
        teacherlist = teacher.split(":")
        for teachers in teacherlist:
            dictTeacherTime[teachers] = dictTeacherTime.get(teachers,[1]*12)
    else:
        dictTeacherTime[teacher] = dictTeacherTime.get(teacher,[1]*12)

In [348]:
# 建立每个key和value的联系
# 比如 课程 和 老师
import numpy as np
import re 

def buildConnectionDict(data, key, value):
    dictKeyValue = {}
    for i in range(len(data)):
        keyName = data.loc[i][key]
        dictKeyValue[keyName] = dictKeyValue.get(keyName,[])
        valueName = data.loc[i][value]
        if valueName is np.nan:
            continue
        elif ":" in valueName:
            valueList = valueName.split(":")
            for values in valueList:
                dictKeyValue[keyName].append(values)
        else:
            dictKeyValue[keyName].append(valueName)
    for key,value in dictKeyValue.items():
        value = list(set(value))
        dictKeyValue[key] =  value
    return dictKeyValue

In [349]:
# 建立所有学院和老师的联系
dataCollege = pd.read_csv("class.csv", usecols=["ci_class_dep","ci_teacher_name"])
dictCollegeTeacher = buildConnectionDict(dataCollege, "ci_class_dep","ci_teacher_name")
# 建立课程和学院的联系
dataCourse = pd.read_csv("class.csv", usecols=["ci_class_dep","ci_course_no"])
dictCourseCollege = buildConnectionDict(dataCourse,"ci_course_no","ci_class_dep")
# 建立课程和老师的联系
dataCourseTeacher = pd.read_csv("class.csv", usecols=["ci_course_no","ci_teacher_name"])
dictCourseTeacher = buildConnectionDict(dataCourseTeacher,"ci_course_no","ci_teacher_name")

In [350]:
%%time
# 假设总共考3天，每天4个考试时间段
# 生成一个字典，对应的形式是这样的{"班级1":[],"班级2":[],"班级3":[],...}
dictClassTime = {}
for courseNumber in SamplingCourse("class.csv"):
    course = courseNumber[0]
    # 查找每一个考这门课的班级
    temp = [1]*12
    tempTeacher = [1] * 12
    classWhoTakeCourse = []
    classCourseNumber = []
    unitClass = []
    classTeacherList = []
    for i in range(len(data)):
        if data.loc[i]["ci_course_no"] == course:
            className = data.loc[i]["ci_class_name"]
            classNumber = data.loc[i]["ci_student_number"]
            # 保存班级人数和班级的时候以整体为单位
            classWhoTakeCourse.append(className)
            classCourseNumber.append(classNumber)

            # 维护每个班级的时间表
            splitClassList = unitedClass(className) 
            for className in splitClassList:
                # 对考这门课的班级的时间进行初始化, 如果不存在就生成一个[1,1,1,1...,]
                dictClassTime[className] = dictClassTime.get(className,[1]*12)
                # 找到所有参加这门考试 班级的共同空闲时间
                # [0,1,1,...,] 和 [1,0,1,...,] 相与
                temp = findCommonTime(temp,dictClassTime[className])
                unitClass.append(className)
            
            classTeacher = data.loc[i]["ci_teacher_name"]
            classTeacherList.append(classTeacher)
            
            # 接下取监考老师的公共时间
            unitTeacher = []
            classTeacher = data.loc[i]["ci_teacher_name"] 
            if ":" in classTeacher:
                teachersList = classTeacher.split(":")
                for teacher in teachersList:
                    tempTeacher = findCommonTime(tempTeacher, dictTeacherTime[teacher])
                    unitTeacher.append(teacher)
            else:
                tempTeacher = findCommonTime(tempTeacher,dictTeacherTime[classTeacher])
                unitTeacher.append(classTeacher)
    
    unitTeacher = list(set(unitTeacher))
    # 两个公共时间相与
    temp = findCommonTime(tempTeacher,temp)
    
    # 判断监考老师是否足够, 不够从同学院找
    if len(unitTeacher) < courseNumber[1]:
        college = dictCourseCollege[course]
        collegeTeachers = dictCollegeTeacher[college[0]]
        for Teacher in collegeTeachers:
            if Teacher not in unitTeacher and findCommonTime(dictTeacherTime[Teacher], tempTeacher) == tempTeacher:
                unitTeacher.append(Teacher)

    # 监考老师多了, 就选到只有这么多老师
    # 非常粗糙            
    if len(unitTeacher) > courseNumber[1]:
        unitTeacher = unitTeacher[0:courseNumber[1]]

    if 1 not in temp:
        print("当前" + course +"无法排考")
    # 找到共同可行的时间, 写入csv, 并且标记占用时间
    else:
        # newClassWhoTakeCourse, newClassCourseNumber = splitClass(classWhoTakeCourse,classCourseNumber)
        writeToCsv(temp,course,classWhoTakeCourse,classCourseNumber, unitTeacher)
        updateClassTime(dictClassTime, unitClass, temp.index(1))
        updateClassTime(dictTeacherTime, unitTeacher, temp.index(1))

Wall time: 9.45 s


In [267]:
# 开始分配老师
dataTemp = pd.read_csv("temp.csv", usecols=["课程","考试时间"])
Monitor = []
for i in range(len(dataTemp)):
    course = dataTemp.loc[i]["课程"]
    time = dataTemp.loc[i]["考试时间"]
    teacherlist = dictCourseTeacher[course]
    for teacher in teacherlist:
        # 如果主课老师时间没问题
        if dictTeacherTime[teacher][time] == 1:
            Monitor.append(teacher)
            updateClassTime(dictTeacherTime,[teacher],time)
            break 
    # 老师不够 请同学院的老师帮忙监考
    # 找这个课程对应的学院
    college = dictCourseCollege[course]
    for teachers in dictCollegeTeacher[college[0]]:
        if dictTeacherTime[teachers][time] == 1:
            Monitor.append(teachers)
            updateClassTime(dictTeacherTime,[teachers],time)
            break
        else:
            continue
    print("无足够老师" + course)

无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师550GL003
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师Z521KB4009
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL001
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师550GL005
无足够老师510GY017
无足够老师510GY017
无足够老师510GY017
无足够老师510GY017
无足够老师510GY017
无足够老师510GY017
无足够老师510GY017
无足够老师510GY017
无足够老师5

In [ ]:
colle